# LearnPlatform COVID-19 Impact on Digital Learning ⌨

# First, we made 3 hypotheses and check whether our hypotheses are correct or not <br>
1. Students will study weekday more than weekends
2. Top 2 platforms are not affected by region
3. The more black/hispanic students are, the lower pptotalraw value

## Import the necessary libraries 📘


In [ ]:
import os
import pandas as pd
import numpy as np
import glob
import plotly.express as px 
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from datetime import date

## Setting path ✋

In [ ]:
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    district_id = filename.split("/")[4].split(".")[0]
    df["district_id"] = district_id
    li.append(df)
    
engagement_df = pd.concat(li)
engagement_df = engagement_df.reset_index(drop=True)
engagement_df.head()

## Data Definition 💋

## Product information data
The product file ```products_info.csv``` includes information about the characteristics of the top 372 products with most users in 2020. The categories listed in this file are part of LearnPlatform's product taxonomy. 

Some products may not have labels due to being duplicate, lack of accurate url or other reasons.

| Name                       | Description                                                                                                                                                                                                                                                                                                                    |
|----------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| LP ID                      | The unique identifier of the product                                                                                                                                                                                                                                                                                           |
| URL                        | Web Link to the specific product                                                                                                                                                                                                                                                                                               |
| Product Name               | Name of the specific product                                                                                                                                                                                                                                                                                                   |
| Provider/Company Name      | Name of the product provider                                                                                                                                                                                                                                                                                                   |
| Sector(s)                  | Sector of education where the product is used                                                                                                                                                                                                                                                                                  |
| Primary Essential Function | The basic function of the product. There are two layers of labels here. Products are first labeled as one of these three categories: LC = Learning & Curriculum, CM = Classroom Management, and SDO = School & District Operations. Each of these categories have multiple sub-categories with which the products were labeled |
|                            | 

In [ ]:
products_df = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
products_df.head(10)

## District information data

The district file districts_info.csv includes information about the characteristics of school districts, including data from NCES (2018-19), FCC (Dec 2018), and Edunomics Lab. In this data set, the identifiable information about the school districts were removed. 

An open source tool ARX (Prasser et al. 2020) was used to transform several data fields and reduce the risks of re-identification. For data generalization purposes some data points were released with a range where the actual value falls under. 

Additionally, there are many missing data marked as 'NaN' indicating that the data was suppressed to maximize anonymization of the dataset.

| Name                   | Description                                                                                                                                                                                                                                                                              |
|------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| district_id            | The unique identifier of the school district                                                                                                                                                                                                                                             |
| state                  | The state where the district resides in                                                                                                                                                                                                                                                  |
| locale                 | NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural. See Locale Boundaries User's Manual for more information.                                                                                                          |
| pct_black/hispanic     | Percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data                                                                                                                                                                                       |
| pct_free/reduced       | Percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data                                                                                                                                                                              |
| countyconnectionsratio | ratio (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version). See FCC data for more information.                                                                         |
| pptotalraw             | Per-pupil total expenditure (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project. The expenditure data are school-by-school, and we use the median value to represent the expenditure of a given school district. |
                                                         

In [ ]:
districts_df = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
districts_df.head(10)

## Engagement data
The engagement data are aggregated at school district level, and each file in the folder engagement_data represents data from one school district.<br> 
The 4-digit file name represents district_id which can be used to link to district information in district_info.csv.<br>
The lp_id can be used to link to product information in product_info.csv.



| Name             | Description                                                                                                    |
|------------------|----------------------------------------------------------------------------------------------------------------|
| time             | date in "YYYY-MM-DD"                                                                                           |
| lp_id            | The unique identifier of the product                                                                           |
| pct_access       | Percentage of students in the district have at least one page-load event of a given product and on a given day |
| engagement_index | Total page-load events per one thousand students of a given product and on a given day                         |

In [ ]:
engagement_df.head(10)

# EDA 📊

# Hypothesis 1 - Students will study weekday more than weekends
## At first, divide the data into weekday and weekends by using datetime

In [ ]:
engagement_df['time']

In [ ]:
engagement_df['day'] = pd.to_datetime(engagement_df['time']).dt.weekday

In [ ]:
engagement_df['day']

In [ ]:
engagement_df['study_yn'] = engagement_df['pct_access'] >0
engagement_df

In [ ]:
def is_weekend():
    return d>4

In [ ]:
engagement_df['weekend'] = engagement_df['day'] > 4 
engagement_df['weekend']

In [ ]:
engagement_df['weekend'].value_counts()
engagement_df

In [ ]:
engagement_df[engagement_df['weekend']==False]['study_yn'].value_counts()

In [ ]:
engagement_df[engagement_df['weekend']==False]['study_yn'].value_counts()/5

In [ ]:
engagement_df[engagement_df['weekend']==True]['study_yn'].value_counts()/2

## For getting mean values of day, I split 5 for weekday, 2 for weekends

In [ ]:
weekstudy = engagement_df[engagement_df['weekend']==False]['study_yn'].value_counts()/5 

In [ ]:
weekendstudy = engagement_df[engagement_df['weekend']==True]['study_yn'].value_counts()/2

In [ ]:
weekstudy

In [ ]:
data = {'week_study': weekstudy[1], 'week_not_study': weekstudy[0], 'weekend_study':weekendstudy[1], 'weekend_not_study':weekendstudy[0]}

In [ ]:
data

In [ ]:
fig = px.bar( x = data.keys() , y = data.values())
fig.update_layout(title_text='Comparison of learning platform utilization between weekday and weekend')
fig.show()

## 1st Hypothesis is correct ✔

# Hypothesis 2 - Top 2 platforms are not affected by region

In [ ]:
engagement_df['pct_access'].describe()

## First, we put 'pct_access' is more than 80% 

In [ ]:
engagement_df[engagement_df['pct_access']>80]

In [ ]:
engagement_df[engagement_df['pct_access']>80].nunique()

In [ ]:
best_pct_access = engagement_df.sort_values(by = 'pct_access', ascending=False)

In [ ]:
best_pct_access.head(100)['lp_id'].unique()

## Among these results, there were 2 URL overlapped. So there are 8 data

In [ ]:
products_df[products_df['LP ID'].isin(best_pct_access.head(10)['lp_id'].unique())] 

## Making mean values for avoiding anomaly data

In [ ]:
engagement_df['pct_access'].groupby(engagement_df['lp_id']).mean().describe() 

In [ ]:
engagement_df[engagement_df['study_yn']==True]['pct_access'].groupby(engagement_df['lp_id']).mean().describe()

In [ ]:
engagement_df[engagement_df['study_yn']==True]['pct_access'].groupby(engagement_df['lp_id']).mean().reset_index().sort_values(by= 'pct_access', ascending = False)[:10]

In [ ]:
df_m1 = engagement_df[engagement_df['study_yn']==True]['pct_access'].groupby(engagement_df['lp_id']).mean().reset_index().sort_values(by= 'pct_access', ascending = False)[:10]

In [ ]:
df_m1

## Merging files 🗂

In [ ]:
result_m1 = pd.merge(df_m1, products_df, how = 'left', left_on = 'lp_id', right_on = 'LP ID')

In [ ]:
result_m1

In [ ]:
pop_platform = result_m1.dropna()

In [ ]:
pop_platform

## Regardless of region, this graph represents the popularity of platforms

In [ ]:
fig = px.bar(pop_platform, x = 'Product Name', y = 'pct_access', color = 'pct_access')
fig.update_layout(title_text='The most popular learning platform')
             
fig.show()

## Next step is defining preference of platform in each region 

In [ ]:
districts_df['locale'].unique()

In [ ]:
df_2 = pd.DataFrame(index = ['1st','2nd','3rd'], columns = ['Product Name','pct_access'])
df_2

In [ ]:
df_2.loc['1st']

In [ ]:
districts_df[['district_id','state','locale']].dropna()

In [ ]:
engagement_df

In [ ]:
engagement_df['district_id'] = engagement_df['district_id'].astype('int64')

## Then, we need to merge 2 data

In [ ]:
engagement_loc_df = pd.merge(engagement_df, districts_df[['district_id', 'state', 'locale']], how = 'left') 
engagement_loc_df

In [ ]:
engagement_loc_df = pd.merge(engagement_loc_df, products_df[['LP ID', 'Product Name']], how='left', left_on='lp_id', right_on='LP ID')

In [ ]:
engagement_loc_df

In [ ]:
ts_by_district = pd.DataFrame(engagement_loc_df[engagement_loc_df['study_yn']==True]['pct_access'].groupby([engagement_loc_df['locale'], engagement_loc_df['Product Name']]).mean())
ts_by_district

## Using for loop in order not to repeat making DataFrame

In [ ]:
district_name = ['Suburb', 'Rural', 'City', 'Town']

for name in district_name:
   globals()['{}_df'.format(name)] = pd.DataFrame(index = ['1st', '2nd', '3rd'], columns = ['Product Name', 'pct_access'])
   loc = ts_by_district.loc[name].sort_values(by = 'pct_access', ascending=False).head(3)
   globals()['{}_df'.format(name)].loc['1st']['Product Name'] = loc.index[0]
   globals()['{}_df'.format(name)].loc['2nd']['Product Name'] = loc.index[1]
   globals()['{}_df'.format(name)].loc['3rd']['Product Name'] = loc.index[2]

   globals()['{}_df'.format(name)].loc['1st']['pct_access'] = loc['pct_access'][0]
   globals()['{}_df'.format(name)].loc['2nd']['pct_access'] = loc['pct_access'][1]
   globals()['{}_df'.format(name)].loc['3rd']['pct_access'] = loc['pct_access'][2]
    

In [ ]:
ts_by_district.loc['City'].sort_values(by = 'pct_access', ascending=False).head(3)

In [ ]:
Suburb_df

## Visualization our results

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(12, 10)

sns.barplot(data=Suburb_df, x="Product Name", y="pct_access", ax=axes[0][0])
sns.barplot(data=Town_df, x="Product Name", y="pct_access", ax=axes[0][1])
sns.barplot(data=City_df, x="Product Name", y="pct_access", ax=axes[1][0])
sns.barplot(data=Rural_df, x="Product Name", y="pct_access", ax=axes[1][1])

ax=axes[0][0].set(ylabel='Access percentage',title="Suburb Top learning platform")
ax=axes[0][1].set(ylabel='Access percentage',title="Town Top learning platform")
ax=axes[1][0].set(ylabel='Access percentage', title="City Top learning platform")
ax=axes[1][1].set(ylabel='Access percentage', title="Rural Top learning platform")




## 2nd Hypothesis is correct ✔

# Hypothesis 3 - The more black/hispanic students, the lower pptotalraw value

In [ ]:
list_1 = districts_df['pct_black/hispanic'].dropna().str.strip('[]').str.split(',').to_list()
list_1


In [ ]:
[float(x) + float(y) for x,y in list_1]

In [ ]:
[round(float(x) + float(y),2)for x,y in list_1]

In [ ]:
districts_df.dropna()

In [ ]:
bh_pp_df = districts_df.dropna(subset =['pp_total_raw', 'pct_free/reduced']).copy()
bh_pp_df

## Make invalid data useful

In [ ]:
black_hispanic_li = bh_pp_df['pct_black/hispanic'].str.strip('[]').str.split(',').to_list()
black_hispanic_li

In [ ]:
[round(float(x) + float(y),2) for x,y in black_hispanic_li]

In [ ]:
black_hispanic_li2 = [round(float(x) + float(y),2) for x,y in black_hispanic_li]

In [ ]:
bh_pp_df['pct_black/hispanic'] = black_hispanic_li2 
bh_pp_df

In [ ]:
pp_total_raw_li = bh_pp_df['pp_total_raw'].str.strip('[]').str.split(',').to_list()
pp_total_raw_li

In [ ]:
[round(float(x) + float(y),2) for x,y in pp_total_raw_li]
pp_total_raw_li2 = [round(float(x) + float(y),2) for x,y in pp_total_raw_li]

In [ ]:
bh_pp_df['pp_total_raw'] = pp_total_raw_li2 
bh_pp_df

In [ ]:
bh_pp_df.drop(['county_connections_ratio'], axis =1, inplace = True) # inplace = True 가 삭제한 데이터 값을 저장해준다 


In [ ]:
bh_pp_df

In [ ]:
pct_free_reduced_li = bh_pp_df['pct_free/reduced'].str.strip('[]').str.split(',').to_list()
pct_free_reduced_li

In [ ]:
[round(float(x) + float(y),2) for x,y in pct_free_reduced_li]
pct_free_reduced_li2 = [round(float(x) + float(y),2) for x,y in pct_free_reduced_li]


In [ ]:
bh_pp_df['pct_free/reduced'] = pct_free_reduced_li2 
bh_pp_df

## Figure out correlation of data

In [ ]:
bh_pp_df[['pct_black/hispanic', 'pct_free/reduced', 'pp_total_raw']].corr(method='pearson')

In [ ]:
mask = np.zeros_like(bh_pp_df[['pct_black/hispanic', 'pp_total_raw', 'pct_free/reduced']].corr(method='pearson'))
mask[np.triu_indices_from(mask)] = True

sns.heatmap(bh_pp_df[['pct_black/hispanic', 'pp_total_raw', 'pct_free/reduced']].corr(method='pearson'),
           cmap = 'RdYlBu_r',
           annot = True,  
           mask=mask,      
           linewidths=.5,  
           cbar_kws={"shrink": .5},
           vmin = -1,vmax = 1  
          )  
plt.show()

## 3rd Hypothesis is correct ✔